In [1]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
import tensorflow_recommenders as tfrs

In [3]:
# Ratings data.
ratings = tfds.load("movie_lens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movie_lens/100k-movies", split="train")

Shuffling and writing examples to /home/linchenxiao/tensorflow_datasets/movie_lens/100k-ratings/0.1.0.incompleteLE0N8V/movie_lens-train.tfrecord


Dataset movie_lens downloaded and prepared to /home/linchenxiao/tensorflow_datasets/movie_lens/100k-ratings/0.1.0. Subsequent calls will reuse this data.


Shuffling and writing examples to /home/linchenxiao/tensorflow_datasets/movie_lens/100k-movies/0.1.0.incompletePUIA53/movie_lens-train.tfrecord


Dataset movie_lens downloaded and prepared to /home/linchenxiao/tensorflow_datasets/movie_lens/100k-movies/0.1.0. Subsequent calls will reuse this data.


# Candidate generation

In [7]:
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)
for x in movies.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}
{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


In [8]:
# 
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

In [11]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [14]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

## Model

In [17]:
embedding_dimension = 32

In [18]:
# Query towel
user_model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.StringLookup(
      vocabulary=unique_user_ids),
  # We add 2 to account for unknown and mask tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 2, embedding_dimension)
])

In [20]:
# Candidate towel
movie_model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.StringLookup(
      vocabulary=unique_movie_titles),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 2, embedding_dimension)
])

In [21]:
# Loss metrics
# To figure out how good our model is, we need to compare the affinity score that the model calculates for this pair to the scores of all the other possible candidates: 
# if the score for the positive pair is higher than for all other candidates, our model is highly accurate.
# Pair-wise loss
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

In [22]:
# Metric
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [23]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

In [25]:
# Training
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 11s 1s/step - factorized_top_k: 0.1071 - factorized_top_k/top_1_categorical_accuracy: 0.0020 - factorized_top_k/top_5_categorical_accuracy: 0.0174 - factorized_top_k/top_10_categorical_accuracy: 0.0376 - factorized_top_k/top_50_categorical_accuracy: 0.1757 - factorized_top_k/top_100_categorical_accuracy: 0.3031 - loss: 66052.5405 - regularization_loss: 0.0000e+00 - total_loss: 66052.5405
Epoch 2/3
10/10 [==============================] - 11s 1s/step - factorized_top_k: 0.1273 - factorized_top_k/top_1_categorical_accuracy: 0.0022 - factorized_top_k/top_5_categorical_accuracy: 0.0242 - factorized_top_k/top_10_categorical_accuracy: 0.0514 - factorized_top_k/top_50_categorical_accuracy: 0.2114 - factorized_top_k/top_100_categorical_accuracy: 0.3473 - loss: 64962.6371 - regularization_loss: 0.0000e+00 - total_loss: 64962.6371
Epoch 3/3
10/10 [==============================] - 12s 1s/step - factorized_top_k: 0.1359 - factorized_top_k/top_1_c

In [26]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 2s 471ms/step - factorized_top_k: 0.0605 - factorized_top_k/top_1_categorical_accuracy: 4.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0046 - factorized_top_k/top_10_categorical_accuracy: 0.0115 - factorized_top_k/top_50_categorical_accuracy: 0.0906 - factorized_top_k/top_100_categorical_accuracy: 0.1955 - loss: 31421.1543 - regularization_loss: 0.0000e+00 - total_loss: 31421.1543


{'factorized_top_k': array([0.0004 , 0.00455, 0.01145, 0.09065, 0.19555], dtype=float32),
 'factorized_top_k/top_1_categorical_accuracy': 0.00039999998989515007,
 'factorized_top_k/top_5_categorical_accuracy': 0.004550000187009573,
 'factorized_top_k/top_10_categorical_accuracy': 0.01145000010728836,
 'factorized_top_k/top_50_categorical_accuracy': 0.09064999967813492,
 'factorized_top_k/top_100_categorical_accuracy': 0.19554999470710754,
 'loss': 28496.6640625,
 'regularization_loss': 0,
 'total_loss': 28496.6640625}

In [27]:
# Create a model that takes in raw query features, and
index = tfrs.layers.ann.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index(movies.batch(100).map(model.movie_model), movies)

# Get recommendations.
_, titles = index(tf.constant(["42"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

Recommendations for user 42: [b'Rent-a-Kid (1995)' b'Little Big League (1994)'
 b'Affair to Remember, An (1957)']
